In [1]:
from transformers import RobertaConfig, RobertaTokenizer, RobertaForMaskedLM, pipeline
import torch

model = RobertaForMaskedLM.from_pretrained("microsoft/codebert-base-mlm")
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base-mlm")

CODE = "def returnInt() -> <mask>:"
fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer)

outputs = fill_mask(CODE)
outputs

[{'score': 0.2260781228542328,
  'token': 6979,
  'token_str': ' int',
  'sequence': 'def returnInt() -> int:'},
 {'score': 0.15783804655075073,
  'token': 5053,
  'token_str': ' Any',
  'sequence': 'def returnInt() -> Any:'},
 {'score': 0.1569615751504898,
  'token': 9291,
  'token_str': ' None',
  'sequence': 'def returnInt() -> None:'},
 {'score': 0.12274446338415146,
  'token': 1907,
  'token_str': ' type',
  'sequence': 'def returnInt() -> type:'},
 {'score': 0.06580942869186401,
  'token': 1666,
  'token_str': '...',
  'sequence': 'def returnInt() ->...:'}]

In [2]:
class CustomModel(torch.nn.Module):
    def __init__(self, model, d, vocabulary_size = 50265): 
        super(CustomModel, self).__init__() 
        self.d = d
        self.model = model
        self.config = model.config
        self.layer = torch.nn.Linear(vocabulary_size, d)
    
    def forward(self, input_ids=None, attention_mask=None):
        model_output = self.model.forward(input_ids=input_ids, attention_mask=attention_mask)        
        final_output_tensor = self.layer.forward(model_output[0])
        model_output.logits = final_output_tensor
        
        return model_output

In [3]:
nl_tokens=tokenizer.tokenize("")
code_tokens=tokenizer.tokenize("def returnInt() ->")
tokens=[tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]+code_tokens+[tokenizer.sep_token]
tokens_ids=tokenizer.convert_tokens_to_ids(tokens)
model(torch.tensor(tokens_ids)[None,:])[0].size()

torch.Size([1, 8, 50265])

In [4]:
custom_model = CustomModel(model, 8)
custom_model.forward(torch.tensor(tokens_ids)[None,:])



MaskedLMOutput(loss=None, logits=tensor([[[-0.5659,  2.8974,  1.7297,  0.6070, -3.9746, -2.7266, -1.6069,
          -1.5802],
         [-2.5298,  0.4019,  0.2934,  3.0135, -3.0737, -3.9806, -2.1049,
          -3.0578],
         [-0.6079,  0.6877, -0.8915,  2.6663, -2.9882, -3.1302, -2.8738,
          -1.5893],
         [-0.8444,  3.2024,  1.6605, -1.4226, -4.5636, -1.7806, -2.8512,
          -2.1173],
         [-0.1082,  1.1586, -2.4872, -0.8006, -4.0596, -2.2575, -3.3089,
          -2.6956],
         [-0.9187,  2.8969, -0.5651,  4.9787,  0.2178, -3.7416, -3.9871,
          -3.8240],
         [ 1.5083,  1.5598,  0.7495, -0.4502, -4.2491, -2.4973, -1.6479,
          -2.7352],
         [ 1.0187,  1.7994, -1.1320, -2.1037, -2.7119, -2.5388, -2.2533,
          -2.4799]]], grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [5]:
custom_model = CustomModel(model, 8)
fill_mask = pipeline('fill-mask', model=custom_model, tokenizer=tokenizer)

outputs = fill_mask(CODE)
outputs

[{'score': 0.3408574163913727,
  'token': 4,
  'token_str': '.',
  'sequence': 'def returnInt() ->.:'},
 {'score': 0.27321097254753113,
  'token': 6,
  'token_str': ',',
  'sequence': 'def returnInt() ->,:'},
 {'score': 0.13929034769535065,
  'token': 1,
  'token_str': '<pad>',
  'sequence': 'def returnInt() ->:'},
 {'score': 0.10849028825759888,
  'token': 5,
  'token_str': ' the',
  'sequence': 'def returnInt() -> the:'},
 {'score': 0.07683997601270676,
  'token': 2,
  'token_str': '</s>',
  'sequence': 'def returnInt() ->:'}]

In [6]:
class TripletLoss(torch.nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin
        
    def calc_euclidean(self, x1, x2):
        return (x1 - x2).pow(2).sum(1)
    
    def forward(self, anchor: torch.Tensor, positive: torch.Tensor, negative: torch.Tensor) -> torch.Tensor:
        distance_positive = self.calc_euclidean(anchor, positive)
        distance_negative = self.calc_euclidean(anchor, negative)
        losses = torch.relu(distance_positive - distance_negative + self.margin)

        return losses.mean()

In [7]:
from tqdm.notebook import tqdm

def tokenize_code(code):
    nl_tokens = tokenizer.tokenize("")
    code_tokens = tokenizer.tokenize(code)
    tokens = [tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]+code_tokens+[tokenizer.sep_token]
    tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
    return torch.tensor(tokens_ids)[None,:]

epochs = 1
data = [("def returnInt() -> <mask>:", "def calcInt() -> <mask>:","def returnFloat() -> <mask>:", "int")]

optimizer = torch.optim.Adam(custom_model.parameters(), lr=0.001)
criterion = torch.jit.script(TripletLoss())

for epoch in tqdm(range(epochs), desc="Epochs"):
    custom_model.train()
    running_loss = []
    for step, (t_a, t_p, t_n, anchor_label) in enumerate(data):
        
        optimizer.zero_grad()
        anchor_out = custom_model(tokenize_code(t_a))
        positive_out = custom_model(tokenize_code(t_p))
        negative_out = custom_model(tokenize_code(t_n))
        
        loss = criterion(anchor_out[0], positive_out[0], negative_out[0])
        loss.backward()
        optimizer.step()

Epochs:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
fill_mask = pipeline('fill-mask', model=custom_model, tokenizer=tokenizer)

outputs = fill_mask(CODE)
outputs

[{'score': 0.4581899344921112,
  'token': 6,
  'token_str': ',',
  'sequence': 'def returnInt() ->,:'},
 {'score': 0.426839143037796,
  'token': 4,
  'token_str': '.',
  'sequence': 'def returnInt() ->.:'},
 {'score': 0.05817270651459694,
  'token': 5,
  'token_str': ' the',
  'sequence': 'def returnInt() -> the:'},
 {'score': 0.05054251477122307,
  'token': 7,
  'token_str': ' to',
  'sequence': 'def returnInt() -> to:'},
 {'score': 0.006245944183319807,
  'token': 0,
  'token_str': '<s>',
  'sequence': 'def returnInt() ->:'}]

In [9]:
import numpy as np
from annoy import AnnoyIndex
from torch.utils.data import DataLoader
import random

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
KNN_TREE_SIZE = 20

def compute_validation_loss_dsl():
    with torch.no_grad():
        # model.eval()
        computed_embed_batches_train = []
        computed_embed_labels_train = []
        # for step, (t_a, t_p, t_n, anchor_label) in enumerate(DataLoader(data)):
        output = custom_model.forward(torch.tensor(tokens_ids)[None,:])
        computed_embed_batches_train.append(output.logits.cpu().numpy())
        # computed_embed_labels_train.append(t_a[1].logits.cpu().numpy())
             
        annoy_index = create_knn_index(np.vstack(computed_embed_batches_train), None, computed_embed_batches_train[0].shape[1])
    return annoy_index

def create_knn_index(train_types_embed: np.array, valid_types_embed: np.array, type_embed_dim:int) -> AnnoyIndex:
    """
    Creates KNNs index for given type embedding vectors, taken from Type4Py
    """

    annoy_idx = AnnoyIndex(type_embed_dim, 'euclidean')

    for i, v in enumerate(tqdm(train_types_embed, total=len(train_types_embed), desc="KNN index")):
        annoy_idx.add_item(i, v[0])

    # if valid_types_embed is not None:
    #     for i, v in enumerate(valid_types_embed):
    #         annoy_idx.add_item(len(train_types_embed) + i, v)

    annoy_idx.build(KNN_TREE_SIZE)
    return annoy_idx


In [10]:
annoy_idx = compute_validation_loss_dsl()
print(annoy_idx)

KNN index:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
from collections import defaultdict

def get_test_embedding():
    with torch.no_grad():
        nl_tokens=tokenizer.tokenize("")
        code_tokens=tokenizer.tokenize("def returnInt() ->")
        tokens=[tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]+code_tokens+[tokenizer.sep_token]
        tokens_ids=tokenizer.convert_tokens_to_ids(tokens)
        computed_embed_batches_test = []
        output = custom_model.forward(torch.tensor(tokens_ids)[None,:])
        print(output)
        computed_embed_batches_test.append(output.logits.cpu().numpy())
        test_label = tokenizer.convert_tokens_to_ids(tokenizer("int"))
        print(test_label)
        return computed_embed_batches_test, test_label

def predict_type_embed(types_embed_array: np.array, types_embed_labels: np.array, indexed_knn: AnnoyIndex, k: int):
    """
    Predict type of given type embedding vectors
    """

    pred_types_embed = []
    pred_types_score = []
    for i, embed_vec in enumerate(tqdm(types_embed_array, total=len(types_embed_array), desc="Finding KNNs & Prediction")):
        idx, dist = indexed_knn.get_nns_by_vector(embed_vec[0], k, include_distances=True)
        pred_idx_scores = compute_types_score(dist, idx, types_embed_labels)
        pred_types_embed.append([i for i, s in pred_idx_scores])
        pred_types_score.append(pred_idx_scores)
    
    return pred_types_embed, pred_types_score

def compute_types_score(types_dist: list, types_idx: list, types_embed_labels: np.array):
        types_dist = 1 / (np.array(types_dist) + 1e-10) ** 2
        types_dist /= np.sum(types_dist)
        types_score = defaultdict(int)
        for n, d in zip(types_idx, types_dist):
            types_score[types_embed_labels[n]] += d
        
        return sorted({t: s for t, s in types_score.items()}.items(), key=lambda kv: kv[1], reverse=True)
    
types_embed_array, types_embed_labels = get_test_embedding()
pred_type_embed, pred_type_score = predict_type_embed(np.vstack(types_embed_array), types_embed_labels, annoy_idx, 8)
print(pred_type_embed)
print(pred_type_score)

MaskedLMOutput(loss=None, logits=tensor([[[-1.3829e+00, -6.9919e+00, -9.7727e+00, -3.7792e+01,  1.2982e+00,
           4.6359e-01,  3.0138e+00,  5.0971e-01],
         [-9.3236e-01, -1.7801e+01, -5.5652e+00, -2.7719e+01,  1.7310e+00,
           2.9367e-01,  2.8298e+00,  6.3305e-02],
         [-2.6360e+00, -2.7185e+01,  1.8042e+00, -2.3171e+01,  1.4147e+00,
           6.6703e-01,  2.8328e+00,  8.1314e-01],
         [-1.2476e+00, -1.5901e+01, -9.3729e-01, -1.6233e+01,  1.8503e+00,
           3.8494e-01,  2.2970e+00,  7.9071e-01],
         [-1.7026e+00, -3.0926e+01, -3.0534e-01, -2.7412e+01,  1.6793e+00,
           9.2657e-01,  2.9036e+00,  2.4046e-02],
         [-1.5037e+00, -8.4329e+00, -1.3389e+01, -4.4969e+01,  2.3260e+00,
           8.6609e-01,  4.1157e+00,  5.0640e-03],
         [-1.5597e+00, -1.8816e+01, -6.1387e-01, -1.8858e+01,  1.7532e+00,
           2.2304e-01,  1.9280e+00,  1.0792e+00],
         [-1.9074e+00, -2.2687e+01, -5.1970e+00, -2.6222e+01,  1.9797e+00,
           5.1457

Finding KNNs & Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[3]]
[[(3, 1.0)]]
